# The Annotated Transformer 보다 친절한 트랜스포머 튜토리얼 !

## 0. 데이터 가공

## 1. BPE 토크나이저 학습
- 앞서 가공한 데이터를 활용해 BPE 기반의 토크나이저를 학습시킵니다.

In [ ]:
params = {
    'batch_size': 64,
    'num_epoch': 15,
    'lr': 0.003,
    'dropout': 0.5,
    'min_frequency': 3,
    'max_len': 512,
    
    'vocab_size': 20000,
    'num_layers': 6,
    'num_heads': 6,
    'hidden_dim': 512,
    'ffn_dim': 2048,
}

In [18]:
from tokenizers import BPETokenizer

# Initialize a tokenizer
kor_tokenizer = BPETokenizer()

# Then train it!
kor_tokenizer.train(
    ['train_korean.txt'],
    vocab_size=params['vocab_size'],
    min_frequency=params['min_frequency'],
    special_tokens=['[PAD]', '[SOS]', '[EOS]', '[UNK]'],
    suffix=''
)

NameError: name 'params' is not defined

In [ ]:
# Initialize a tokenizer
eng_tokenizer = BPETokenizer()

# Then train it!
eng_tokenizer.train(
    ['train_english.txt'],
    vocab_size=params['vocab_size'],
    min_frequency=params['min_frequency'],
    special_tokens=['[PAD]', '[SOS]', '[EOS]', '[UNK]'],
    suffix=''
)

In [ ]:
pad_idx = kor_tokenizer.token_to_id('[PAD]')
sos_idx = kor_tokenizer.token_to_id('[SOS]')
eos_idx = kor_tokenizer.token_to_id('[EOS]')

In [ ]:
kor_tokenizer.enable_padding(pad_id=pad_idx, pad_token='[PAD]', max_length=params['max_len'])
eng_tokenizer.enable_padding(pad_id=pad_idx, pad_token='[PAD]', max_length=params['max_len'])

In [ ]:
kor_encoded_data = kor_tokenizer.encode_batch(kor_train_data)
eng_encoded_data = eng_tokenizer.encode_batch(eng_train_data)

## 2. Transformer 모델 구현

- 먼저 모델 구현에 필요한 라이브러리들을 모두 임포트합니다.
- 실험을 함에 있어 항상 실험의 **Reproducibility**를 보장하기 위해 Seed 설정을 해줍니다.

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

from torch.utils import data

torch.manual_seed(32)
torch.cuda.manual_seed(32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class MultiHeadAttention(nn.Module):
    '''멀티 헤드 어텐션 레이어'''
    def __init__(self, params):
        super(MultiHeadAttention, self).__init__()
        assert params['hidden_dim'] % params['num_heads'] == 0, "hidden dimension must be divisible by the number of heads"
        self.num_heads = params['num_heads']
        self.attn_dim = params['hidden_dim'] // self.num_heads
        
        self.q_w = nn.Linear(params['hidden_dim'], self.num_heads * self.attn_dim)
        self.k_w = nn.Linear(params['hidden_dim'], self.num_heads * self.attn_dim)
        self.v_w = nn.Linear(params['hidden_dim'], self.num_heads * self.attn_dim)
        
        self.o_w = nn.Linear(self.num_heads * self.attn_dim, params['hidden_dim'])
        
    def forward(self, q, k, v, mask=None):
        " q, k, v = [배치 사이즈, 문장 길이, 은닉 차원] "
        
        batch_size = q.size(0)
        
        q = self.q_w(q).view(batch_size, -1, self.num_heads, self.attn_dim).transpose(1, 2)
        k = self.k_w(k).view(batch_size, -1, self.num_heads, self.attn_dim).transpose(1, 2)
        v = self.v_w(v).view(batch_size, -1, self.num_heads, self.attn_dim).transpose(1, 2)
        # q, k, v = [배치 사이즈, 헤드 갯수, 문장 길이, 어텐션 차원]
        
        attn = torch.matmul(q, k.transpose(-1, -2))
        # attn = [배치 사이즈, 헤드 갯수, 문장 길이, 문장 길이]
        
        if mask is not None:
            attn.masked_fill(mask==0, -1e9)
        
        score = F.softmax(attn, dim=-1)
        # score = [배치 사이즈, 헤드 갯수, 문장 길이, 문장 길이]
        
        output = torch.matmul(score, v)
        # output = [배치 사이즈, 헤드 갯수, 문장 길이, 어텐션 차원]
        
        output = output.transpose(1, 2).contiguous()
        # output = [배치 사이즈, 문장 길이, 헤드 갯수, 어텐션 차원]
        
        output = output.view(batch_size, -1, self.num_heads * self.attn_dim)
        # output = [배치 사이즈, 문장 길이, 은닉 차원]
        
        output = self.o_w(output)
        # output = [배치 사이즈, 문장 길이, 은닉 차원]
        
        return output, score

In [28]:
def create_src_mask(src):
    " source = [배치 사이즈, 소스 문장 길이] "

    src_len = src.size(1)
    
    src_mask = (src == pad_idx)
    # src_mask = [배치 사이즈, 소스 문장 길이]
    
    src_mask = src_mask.unsqueeze(1).repeat(1, src_len, 1)
    # src_mask = [배치 사이즈, 소스 문장 길이, 소스 문장 길이]
    
    return src_mask


def create_tgt_mask(src, tgt):
    " src = [배치 사이즈, 소스 문장 길이] "
    " tgt = [배치 사이즈, 타겟 문장 길이] "
    batch_size, tgt_len = tgt.size()
    
    subsequent_mask = torch.triu(torch.ones(tgt_len, tgt_len), diagonal=1)
    # subsequent_mask = [타겟 문장 길이, 타겟 문장 길이]
    
    subsequent_mask = subsequent_mask.unsqueeze(0).repeat(batch_size, 1, 1)
    # subsquent_mask = [배치 사이즈, 타겟 문장 길이, 타겟 문장 길이]
    
    src_mask = (src == pad_idx)
    tgt_mask = (tgt == pad_idx)
    # src_mask = [배치 사이즈, 소스 문장 길이]
    # tgt_mask = [배치 사이즈, 타겟 문장 길이]
    
    src_mask = src_mask.unsqueeze(1).repeat(1, tgt_len, 1)
    tgt_mask = tgt_mask.unsqueeze(1).repeat(1, tgt_len, 1)
    # src_mask = [배치 사이즈, 타겟 문장 길이, 소스 문장 길이]
    # tgt_mask = [배치 사이즈, 타겟 문장 길이, 타겟 문장 길이]
    
    tgt_mask = target_mask | subsequent_mask
    
    return src_mask, tgt_mask

![](img/positionwise.png)

In [13]:
class PositionwiseFeedForward(nn.Module):
    '''포지션 와이즈 피드 포워드 레이어'''
    def __init__(self, parmas):
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(params['hidden_dim'], params['ffn_dim'])
        self.fc2 = nn.Linear(params['ffn_dim'], params['hidden_dim'])
        self.dropout = nn.Dropout(params['dropout'])
    
    def forward(self, x):
        " x = [배치 사이즈, 문장 길이, 은닉 차원] "

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 11)

![](img/pos.png)

In [23]:
class PositionalEncoding(nn.Module):
    def __init__(self, params):
        super(PositionalEncoding, self).__init__()
        sinusoid = np.array([pos / np.power(10000, 2 * i / params['hidden_dim'])
                            for pos in range(params['max_len']) for i in range(params['hidden_dim'])])
        # sinusoid = [문장 최대 길이 * 은닉 차원]

        sinusoid = sinusoid.reshape(params['max_len'], -1)
        # sinusoid = [문장 최대 길이, 은닉 차원]

        sinusoid[:, 0::2] = np.sin(sinusoid[:, 0::2])
        sinusoid[:, 1::2] = np.cos(sinusoid[:, 1::2])
        sinusoid = torch.FloatTensor(sinusoid).to(device)

        self.embedding = nn.Embedding.from_pretrained(sinusoid, freeze=True)
        
    def forward(self, x):
        " x = [배치 사이즈, 문장 길이] "
        
        pos = torch.arange(x.size(-1), dtype=torch.long).to(device)
        # pos = [배치 사이즈, 문장 길이]

        embed = self.embedding(pos)
        # embed = [배치 사이즈, 문장 길이, 은닉 차원]
        return embed

![](img/encoder.png)

In [9]:
class EncoderLayer(nn.Module):
    '''인코더 레이어'''
    def __init__(self, params):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(params)
        self.layer_norm1 = nn.LayerNorm(params['hidden_dim'])
        self.feed_forward = PositionwiseFeedForward(params)
        self.layer_norm2 = nn.LayerNorm(params['hidden_dim'])
        self.dropout = nn.Dropout(params['dropout'])
        
    def forward(self, x, src_mask):
        " x = [배치 사이즈, 문장 길이, 은닉 차원] "
        
        residual = x
        x = self.self_attn(x, x, x, src_mask)
        x = self.dropout(x)
        x = residual + x
        x = self.layer_norm1(x)
        
        residual = x
        x = self.feed_forward(x)
        x = self.dropout(x)
        x = resiudal + x
        x = self.layer_norm2(x)
        
        return x


class Encoder(nn.Module):
    '''트랜스포머 인코더'''
    def __init__(self, params):
        super(Encoder, self).__init__()
        self.tok_embedding = nn.Linear(params['vocab_size'], params['hidden_dim'], padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding()
        self.layers = nn.ModuleList([EncoderLayer(params) for _ in range(params['num_layers'])])
        
    def forward(self, x):
        " x = [배치 사이즈, 문장 길이] "
        
        x = self.tok_embedding(x) + self.pos_embedding(x)
        src_mask = create_source_mask(x)
        
        for layer in self.layers:
            x = layer(x, src_mask)
            
        return x

![](img/decoder.png)

In [19]:
class DecoderLayer(nn.Module):
    '''디코더 레이어'''
    def __init__(self, params):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(params)
        self.layer_norm1 = nn.LayerNorm(params['hidden_dim'])

        self.enc_dec_attn = MultiHeadAttention(params)
        self.layer_norm2 = nn.LayerNorm(params['hidden_dim'])
        self.feed_forward = PositionwiseFeedForward(params)
        self.layer_norm2 = nn.LayerNorm(params['hidden_dim'])
        self.dropout = nn.Dropout(params['dropout'])
        
    def forward(self, x, tgt_mask, enc_output, src_mask):
        " x = [배치 사이즈, 문장 길이, 은닉 차원] "
        
        residual = x
        x = self.self_attn(x, x, x, tgt_mask)
        x = self.dropout(x)
        x = residual + x
        x = self.layer_norm1(x)
        
        residual = x
        x = self.enc_dec_attn(x, enc_output, enc_output, src_mask)
        x = self.dropout(x)
        x = residual + x
        x = self.layer_norm2(x)
        
        residual = x
        x = self.feed_forward(x)
        x = self.dropout(x)
        x = resiudal + x
        x = self.layer_norm3(x)
        
        return x


class Decoder(nn.Module):
    '''트랜스포머 디코더'''
    def __init__(self, params):
        super(Decoder, self).__init__()
        self.tok_embedding = nn.Linear(params['vocab_size'], params['hidden_dim'], padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding()
        self.layers = nn.ModuleList([DecoderLayer(params) for _ in range(params['num_layers'])])
        
    def forward(self, x, enc_output):
        " x = [배치 사이즈, 문장 길이] "
        
        x = self.tok_embedding(x) + self.pos_embedding(x)
        src_mask, tgt_mask = create_target_mask(enc_output, x)
        
        for layer in self.layers:
            x, attn_map = layer(x, tgt_mask, enc_output, src_mask)
            
        x = torch.matmul(target, self.tok_embedding.weight.transpose(0, 1))
        
        return x, attn_map

In [5]:
class Transformer(nn.Module):
    '''트랜스포머 네트워크'''
    def __init__(self, params):
        super(Transformer, self).__init__()
        self.encoder = Encoder(params)
        self.decoder = Decoder(params)
    
    def forward(self, source, target):
        encoder_output = self.encoder(source)
        decoder_output, attn_map = self.decoder(target, encoder_output)
        return decoder_output, attn_map
    
    def count_params(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

![](img/optim.png)

In [21]:
class ScheduledOptim:
    def __init__(self, optimizer, warmup_steps):
        self.init_lr = np.power(params['hidden_dim'], -0.5)
        self.optimizer = optimizer
        self.step_num = 0
        self.warmup_steps = warmup_steps
    
    def step(self):
        self.step_num += 1
        lr = self.init_lr * self.get_scale()
        
        for p in self.optimizer.param_gropus:
            p['lr'] = lr
            
        self.optimizer.step()
    
    def zero_grad(self):
        self.optimizer.zero_grad()
    
    def get_scale(self):
        return np.min([
            np.power(self.step_num, -0.5),
            self.step_num * np.power(self.warmup_steps, -1.5)
        ])

## 3. 모델 학습

In [26]:
model = Transformer(params)
model.to(device)
model.count_params()

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
criterion.to(device)

optimizer = ScheduledOptim(
    optim.Adam(model.parameters(), betas=[0.9, 0.98], eps=1e-9),
    warmup_steps=4000
)

for epoch in params['num_epoch']:
    model.train()
    
    for batch in train_iter:
        pass

NameError: name 'params' is not defined

## 4. 결과 확인

In [16]:
def predict():
    pass

### 참고자료
- [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)
- [jadore801120/attention-is-all-you-need-pytorch](https://github.com/jadore801120/attention-is-all-you-need-pytorch)
- [tunz/transformer-pytorch](https://github.com/tunz/transformer-pytorch)
- [IgorSusmelj/pytorch-styleguide](https://github.com/IgorSusmelj/pytorch-styleguide)


### TODO
- **Beam Search** 디코딩 추가
- **Label Smoothing** 기법 추가